# Preprocessing
## Imports

In [69]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.transforms as transforms
import tarfile
import pandas as pd
from PIL import Image

### Opening Data Files

In [5]:
def open_tar(filename, dest=None):
    with tarfile.open(filename, 'r:gz') as tar:
        if dest is None:
            tar.extractall('./')
        else:
            tar.extractall(dest)
    print("Extraction successful!")

Below extracts into pokemon/icons path

In [15]:
open_tar("./datasource/pokemon-icons.tar.gz")

Extraction successful!


Below extracts into pokemon/main-sprites path

In [16]:
open_tar("./datasource/generation-1.tar.gz")

Extraction successful!


### Create Pokedex

In [20]:
def make_pokedex(path="pokedex/pokemon.csv"):
    pokedex = pd.read_csv(path)
    return pokedex

In [21]:
pokedex = make_pokedex()
print(pokedex)

            Name     Type1   Type2
0      bulbasaur     Grass  Poison
1        ivysaur     Grass  Poison
2       venusaur     Grass  Poison
3     charmander      Fire     NaN
4     charmeleon      Fire     NaN
..           ...       ...     ...
804    stakataka      Rock   Steel
805  blacephalon      Fire   Ghost
806      zeraora  Electric     NaN
807       meltan     Steel     NaN
808     melmetal     Steel     NaN

[809 rows x 3 columns]


In [89]:
pokedex["Name"][50]

'dugtrio'

## Image Augmentation

In [73]:
# Sample image to test augmentations on
img = Image.open("pokedex/images/images/abra.png")
img.show()

In [78]:
def img_rotate(img, rot=None):
    if rot is None:
        rot = np.random.uniform(0, 360)
    
    rot_img = img.rotate(rot)

    rot_img.show(title="Rotated Image")

In [83]:
# Test random rotation
img_rotate(img)

## Organize Samples

Locations with images that are useful:
* pokedex/images/images (high def images)
* pokemon/icons (pixelated images and root of below subfolders. Not all pokemon have them!)
    * pokemon/icons/old (pixelated old images. Not all pokemon have them!)
    * pokemon/icons/right (pixelated images flipped. Not all pokemon have them!)
* pokemon/main-sprites (png for all pixelated gen1 pokemon!)
    * ./red-blue (red/blue version of sprites)
    * ./red-green (red/green version of sprites)
    * ./yellow (yellow version of sprites)

In [90]:
def sample_data(start_id, end_id, pokedex, augment={}):
    for id_ in range(start_id, end_id):
        name = pokedex["Name"][id_]
        print(name)

In [91]:
sample_data(0, 50, pokedex)

bulbasaur
ivysaur
venusaur
charmander
charmeleon
charizard
squirtle
wartortle
blastoise
caterpie
metapod
butterfree
weedle
kakuna
beedrill
pidgey
pidgeotto
pidgeot
rattata
raticate
spearow
fearow
ekans
arbok
pikachu
raichu
sandshrew
sandslash
nidoran-f
nidorina
nidoqueen
nidoran-m
nidorino
nidoking
clefairy
clefable
vulpix
ninetales
jigglypuff
wigglytuff
zubat
golbat
oddish
gloom
vileplume
paras
parasect
venonat
venomoth
diglett
